In [605]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [621]:
# Parameters
S0 = 75                             # Initial stock price
F = 100                             # Face value of the bond
X = 100                             # Strike price for conversion
T = 5                               # Time to maturity (in years)
r = 0.07                            # Risk-free rate
q = 0
b = r - q                           # Cost of carry 
k = 0.03                            # Credit spread
sigma = 0.2                         # Volatility
dt = 1
coupon = 6                          # Coupon payment per year
m = 1                               # Conversion ratio (1 bond = 1 stock)
N = 250                             # Number of time steps
conversion_ratio = F / X            # Conversion ratio

In [ ]:
# Convertible bond pricing model (Cox-Ross-Rubinstein method)
def CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e'):
    # Time step
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))     # Up factor
    d = 1 / u                           # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Create the binomial stock price tree
    StockPrice = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            StockPrice[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Create the bond price tree (including coupon payments)
    BondPrice = np.zeros((N + 1, N + 1))
    # Create the conversion probability tree
    ConversionProbability = np.zeros((N + 1, N + 1))

    # Calculate bond prices and conversion probabilities at maturity
    for j in range(N + 1):
        stock_price = StockPrice[j, N]
        bond_value_at_maturity = max(conversion_ratio * stock_price, F + coupon)
        BondPrice[j, N] = bond_value_at_maturity
        # Conversion probability is 1 if conversion is optimal at maturity
        ConversionProbability[j, N] = 1 if conversion_ratio * stock_price > F + coupon else 0

    # Initialize coupon timing and time management
    CouponTime = T  # Initial coupon time matches maturity

    # Backward calculation for bond price and conversion probability
    for i in range(N - 1, -1, -1):
        if CouponTime != (T - np.ceil((N - i + 1) * dt)):
            CouponPayment = 1  # It's time to pay a coupon
            CouponTime = T - np.ceil((N - i) * dt)
            Time = CouponTime - dt * i
        else:
            CouponPayment = 0

        for j in range(i + 1):
            stock_price = StockPrice[j, i]

            # Dynamic discount factors (ru, rd)
            ru = ConversionProbability[j+1, i+1] * r + (1 - ConversionProbability[j+1, i+1]) * (r + k)
            rd = ConversionProbability[j, i+1] * r + (1 - ConversionProbability[j, i+1]) * (r + k)

            if CouponPayment == 1:
                CouponValue = coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            # Calculate bond price using backward induction
            hold_bond_value = CouponValue + p * BondPrice[j + 1, i + 1] * np.exp(-ru * dt) 
            + (1 - p) * BondPrice[j, i + 1] * np.exp(-rd * dt)
            convert_to_stock_value = conversion_ratio * stock_price

            # Modify for American or European option style
            if AmeEur == 'a':
                # American style: check for early exercise
                BondPrice[j, i] = max(hold_bond_value, convert_to_stock_value)
            else:
                # European style: only exercise at maturity
                BondPrice[j, i] = hold_bond_value

            # Conversion probability: set to 1 if conversion happens, otherwise average the next nodes
            if BondPrice[j, i] == convert_to_stock_value and AmeEur == 'a':
                ConversionProbability[j, i] = 1
            else:
                ConversionProbability[j, i] = p * ConversionProbability[j + 1, i + 1] 
                + (1 - p) * ConversionProbability[j, i + 1]

    return BondPrice[0,0]



In [623]:
MKTPrice=CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, 'e')
print(MKTPrice)

131.7639536868685


In [588]:
# Delta Calculation
def calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump=0.5/100):
    bond_price_up = CRR_convertible_bond(S0 + (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0 - (S0*bump), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate delta
    delta = (bond_price_up - bond_price_down) / (2 * bump * 100)
    return delta

# Calculate Delta
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Delta of the Convertible Bond: {delta:.4f}")


Delta of the Convertible Bond: 0.4076


In [589]:
bump=0.5/100
Taylor_approx = MKTPrice + delta * bump
print(Taylor_approx)
Exact_price=CRR_convertible_bond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, 'e')
print(Exact_price)

131.7659917722593
131.96776222594872


In [590]:
def calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_rho=1/10000):
    bond_price_up = CRR_convertible_bond(S0, F, T, r + bump_rho, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0, F, T, r - bump_rho, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate Rho
    rho = (bond_price_up - bond_price_down) / (2 * bump_rho)
    return rho

rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Rho of the Convertible Bond: {rho:.4f}")

Rho of the Convertible Bond: -224.2896


In [591]:
bump_rho=1/10000
Taylor_approx = MKTPrice + rho * bump_rho
print(Taylor_approx)
Exact_price=CRR_convertible_bond(S0, F, T, r*(1+bump_rho), k, coupon, conversion_ratio, N, sigma, 'e')
print(Exact_price)

131.74152472193688
131.76238372287816


In [592]:
# Function to calculate Vega
def calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_sigma=1/100):
    bond_price_up = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma + bump_sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma - bump_sigma, AmeEur)
    vega = (bond_price_up - bond_price_down) / (2 * bump_sigma)
    return vega

# Calculate Vega
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Vega of the Convertible Bond: {vega:.4f}")


Vega of the Convertible Bond: 69.7186


In [593]:
bump=1/100
Taylor_approx = MKTPrice + vega * bump
print(Taylor_approx)
Exact_price=CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma+bump, 'e')
print(Exact_price)

132.4611396696892
132.84226075839206


In [594]:
# Function to calculate Gamma
def calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_gamma=0.5/100): #medesimo bump del delta.
    bond_price_up = CRR_convertible_bond(S0 + (S0*bump_gamma), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_mid = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_down = CRR_convertible_bond(S0 - (S0*bump_gamma), F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    # Calculate Gamma
    gamma = (bond_price_up - (2 * bond_price_mid) + bond_price_down) / ((S0*bump_gamma) ** 2)
    return gamma

# Calculate Gamma
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Gamma of the Convertible Bond: {gamma:.4f}")

Gamma of the Convertible Bond: 0.0000


In [595]:
bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
print(Taylor_approx)
Exact_price=CRR_convertible_bond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, 'e')
print(Exact_price)

131.7659917722593
131.96776222594872


In [596]:
# Function to calculate Theta
def calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e', bump_theta=1/360):
    bond_price_shorter_time = CRR_convertible_bond(S0, F, T - bump_theta, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    bond_price_current_time = CRR_convertible_bond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur)
    theta = (bond_price_shorter_time - bond_price_current_time) / bump_theta
    return theta

# Calculate Theta
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, AmeEur='e')
print(f"Theta of the Convertible Bond: {theta:.4f}")


Theta of the Convertible Bond: 7.0088


In [597]:
bump=1/360
Taylor_approx=MKTPrice + theta * bump
print(Taylor_approx)
Exact_price=CRR_convertible_bond(S0, F, T-bump, r, k, coupon, conversion_ratio, N, sigma, 'e')
print(Exact_price)

131.7834225337996
131.7834225337996


In [ ]:
# Function to calculate tree parameters (u, d, p) for different models
def calculate_tree_parameters(r, q, sigma, dt, model_type, drift=0):
    if model_type == 'CRR':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(r * dt) - d) / (u - d)
    elif model_type == 'CRR_Drift':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp((r + drift) * dt) - d) / (u - d)
    elif model_type == 'Jarrow-Rudd':
        u = np.exp((r - q - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt))
        d = np.exp((r - q - 0.5 * sigma ** 2) * dt - sigma * np.sqrt(dt))
        p = 0.5
    elif model_type == 'Tian':
        eta=np.exp(dt*sigma**2)
        u=0.5*np.exp(r*dt)*eta*(eta+1+np.sqrt(eta**2+2*eta-3))
        d=0.5*np.exp(r*dt)*eta*(eta+1-np.sqrt(eta**2+2*eta-3))
        p=(np.exp((b)*dt)-d)/(u-d)
    elif model_type == 'Haahtela':
        u = np.exp(r * dt) * (1 + np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        d = np.exp(r * dt) * (1 - np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        p = (np.exp((r - q) * dt) - d) / (u - d)
    else:
        raise ValueError("Unknown model type provided.")
    return u, d, p
    
def LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='', AmeEur='e', q=0, drift=0):
    dt = T / N  # Time step

    # Calculate u, d, p using the specified model type
    u, d, p = calculate_tree_parameters(r, q, sigma, dt, model_type, drift)

    # Create the binomial stock price tree
    StockPrice = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            StockPrice[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Create the bond price tree (including coupon payments)
    BondPrice = np.zeros((N + 1, N + 1))
    # Create the conversion probability tree
    ConversionProbability = np.zeros((N + 1, N + 1))

    # Calculate bond prices and conversion probabilities at maturity
    for j in range(N + 1):
        stock_price = StockPrice[j, N]
        bond_value_at_maturity = max(conversion_ratio * stock_price, F + coupon)
        BondPrice[j, N] = bond_value_at_maturity
        ConversionProbability[j, N] = 1 if conversion_ratio * stock_price > F + coupon else 0

    # Initialize coupon timing and time management
    CouponTime = T  # Initial coupon time matches maturity

    # Backward calculation for bond price and conversion probability
    for i in range(N - 1, -1, -1):
        if CouponTime != (T - np.ceil((N - i + 1) * dt)):
            CouponPayment = 1  # It's time to pay a coupon
            CouponTime = T - np.ceil((N - i) * dt)
            Time = CouponTime - dt * i
        else:
            CouponPayment = 0

        for j in range(i + 1):
            stock_price = StockPrice[j, i]

            # Dynamic discount factors (ru, rd)
            ru = ConversionProbability[j+1, i+1] * r + (1 - ConversionProbability[j+1, i+1]) * (r + k)
            rd = ConversionProbability[j, i+1] * r + (1 - ConversionProbability[j, i+1]) * (r + k)

            if CouponPayment == 1:
                CouponValue = coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            hold_bond_value = CouponValue + p * BondPrice[j + 1, i + 1] * np.exp(-ru * dt) 
            + (1 - p) * BondPrice[j, i + 1] * np.exp(-rd * dt)
            convert_to_stock_value = conversion_ratio * stock_price

            # Modify for American or European option style
            if AmeEur == 'a':  # American option: can convert at any time
                BondPrice[j, i] = max(hold_bond_value, convert_to_stock_value)

            else:  # European option: can only convert at maturity
                BondPrice[j, i] = hold_bond_value

            # Update the conversion probability based on the style
            if AmeEur == 'a' and BondPrice[j, i] == convert_to_stock_value:
                ConversionProbability[j, i] = 1
            else:
                ConversionProbability[j, i] = p * ConversionProbability[j + 1, i + 1] 
                + (1 - p) * ConversionProbability[j, i + 1]

    return BondPrice[0,0]


In [ ]:
# Delta calculation
def calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur='e', bump=0.5/100):
    bond_price_up = LatticeConvertibleBond(S0 + S0 * bump, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    bond_price_down = LatticeConvertibleBond(S0 - S0 * bump, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    delta = (bond_price_up - bond_price_down) / (2 * bump * 100)
    return delta

# Gamma calculation
def calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur='e', bump=0.5/100):
    bond_price_up = LatticeConvertibleBond(S0 + S0 * bump, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    bond_price_mid = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    bond_price_down = LatticeConvertibleBond(S0 - S0 * bump, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    gamma = (bond_price_up - (2 * bond_price_mid) + bond_price_down) / ((S0*bump) ** 2)
    return gamma

# Rho calculation
def calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur='e', bump=1/10000):
    bond_price_up = LatticeConvertibleBond(S0, F, T, r + bump, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    bond_price_down = LatticeConvertibleBond(S0, F, T, r - bump, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    rho = (bond_price_up - bond_price_down) / (2 * bump)
    return rho

# Vega calculation
def calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur='e', bump=1/100):
    bond_price_up = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma + bump, model_type, AmeEur)
    bond_price_down = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma - bump, model_type, AmeEur)
    vega = (bond_price_up - bond_price_down) / (2 * bump)
    return vega

# Theta calculation
def calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur='e', bump=1/360):
    bond_price_shorter_time = LatticeConvertibleBond(S0, F, T - bump, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    bond_price_current_time = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type, AmeEur)
    theta = (bond_price_shorter_time - bond_price_current_time) / bump
    return theta


In [ ]:
# Market Price
MKTPrice = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, 
                                  sigma, model_type='CRR', AmeEur='e')
print(f'CRR:')
print(f"Market Price: {MKTPrice:.10f}")

# Calculate Greeks
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, 
                        model_type='CRR', AmeEur='e')
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, 
                        model_type='CRR', AmeEur='e')
rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, 
                    model_type='CRR', AmeEur='e')
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma,
                      model_type='CRR', AmeEur='e')
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, 
                        model_type='CRR', AmeEur='e')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Rho: {rho:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Theta: {theta:.4f}")

bump=0.5/100
Taylor_approx=MKTPrice + delta * bump
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio,
                                   N, sigma, model_type='CRR', AmeEur='e')
print(f"Δ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: 
      {Exact_price:.10f};")

bump=1/10000
Taylor_approx = MKTPrice + rho * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r*(1+bump), k, coupon, conversion_ratio, 
                                   N, sigma, model_type='CRR', AmeEur='e')
print(f"𝜌:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: 
      {Exact_price:.10f};")

bump=1/100
Taylor_approx = MKTPrice + vega * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, 
                                   sigma+bump, model_type='CRR', AmeEur='e')
print(f"𝜗:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: 
      {Exact_price:.10f};")

bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, 
                                   N, sigma, model_type='CRR', AmeEur='e')
print(f"Γ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: 
      {Exact_price:.10f};")

bump=1/360
Taylor_approx=MKTPrice + theta * bump
Exact_price=LatticeConvertibleBond(S0, F, T-bump, r, k, coupon, conversion_ratio, 
                                   N, sigma, model_type='CRR', AmeEur='e')
print(f"Θ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: 
      {Exact_price:.10f};")


CRR:
Market Price: 131.7639536869
Delta: 0.4076
Gamma: 0.0000
Rho: -224.2896
Vega: 69.7186
Theta: 7.0088
Δ: Taylor Approximation: 131.7659917723; Exact price: 131.9677622259;
𝜌: Taylor Approximation: 131.7415247219; Exact price: 131.7623837229;
𝜗: Taylor Approximation: 132.4611396697; Exact price: 132.8422607584;
Γ: Taylor Approximation: 131.7659917723; Exact price: 131.9677622259;
Θ: Taylor Approximation: 131.7834225338; Exact price: 131.7834225338;


In [601]:
# Market Price
MKTPrice = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
print(f'Tian:')
print(f"Market Price: {MKTPrice:.4f}")

# Calculate Greeks                                                                                           
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Rho: {rho:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Theta: {theta:.4f}")

bump=0.5/100
Taylor_approx=MKTPrice + delta * bump
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
print(f"Δ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/10000
Taylor_approx = MKTPrice + rho * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r*(1+bump), k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
print(f"𝜌:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/100
Taylor_approx = MKTPrice + vega * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma+bump, model_type='Tian', AmeEur='e')
print(f"𝜗:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
print(f"Γ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/360
Taylor_approx=MKTPrice + theta * bump
Exact_price=LatticeConvertibleBond(S0, F, T-bump, r, k, coupon, conversion_ratio, N, sigma, model_type='Tian', AmeEur='e')
print(f"Θ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

Tian:
Market Price: 131.9799
Delta: 0.4176
Gamma: -0.0000
Rho: -564.8080
Vega: 43.1465
Theta: 7.6547
Δ: Taylor Approximation: 131.9819653953; Exact price: 132.1886575393;
𝜌: Taylor Approximation: 131.9233967914; Exact price: 131.9759239768;
𝜗: Taylor Approximation: 132.4113425214; Exact price: 132.4072884689;
Γ: Taylor Approximation: 131.9819653953; Exact price: 132.1886575393;
Θ: Taylor Approximation: 132.0011405128; Exact price: 132.0011405128;


In [602]:
# Market Price
MKTPrice = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
print(f'CRR_Drift:')
print(f"Market Price: {MKTPrice:.4f}")

# Calculate Greeks
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Rho: {rho:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Theta: {theta:.4f}")

bump=0.5/100
Taylor_approx=MKTPrice + delta * bump
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
print(f"Δ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/10000
Taylor_approx = MKTPrice + rho * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r*(1+bump), k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
print(f"𝜌:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/100
Taylor_approx = MKTPrice + vega * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma+bump, model_type='CRR_Drift', AmeEur='e')
print(f"𝜗:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
print(f"Γ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/360
Taylor_approx=MKTPrice + theta * bump
Exact_price=LatticeConvertibleBond(S0, F, T-bump, r, k, coupon, conversion_ratio, N, sigma, model_type='CRR_Drift', AmeEur='e')
print(f"Θ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

CRR_Drift:
Market Price: 131.7640
Delta: 0.4076
Gamma: 0.0000
Rho: -224.2896
Vega: 69.7186
Theta: 7.0088
Δ: Taylor Approximation: 131.7659917723; Exact price: 131.9677622259;
𝜌: Taylor Approximation: 131.7415247219; Exact price: 131.7623837229;
𝜗: Taylor Approximation: 132.4611396697; Exact price: 132.8422607584;
Γ: Taylor Approximation: 131.7659917723; Exact price: 131.9677622259;
Θ: Taylor Approximation: 131.7834225338; Exact price: 131.7834225338;


In [603]:
# Market Price
MKTPrice = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
print(f'Jarrow-Rudd:')
print(f"Market Price Jarrow-Rudd: {MKTPrice:.4f}")

# Calculate Greeks
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Rho: {rho:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Theta: {theta:.4f}")

bump=0.5/100
Taylor_approx=MKTPrice + delta * bump
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
print(f"Δ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/10000
Taylor_approx = MKTPrice + rho * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r*(1+bump), k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
print(f"𝜌:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/100
Taylor_approx = MKTPrice + vega * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma+bump, model_type='Jarrow-Rudd', AmeEur='e')
print(f"𝜗:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
print(f"Γ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/360
Taylor_approx=MKTPrice + theta * bump
Exact_price=LatticeConvertibleBond(S0, F, T-bump, r, k, coupon, conversion_ratio, N, sigma, model_type='Jarrow-Rudd', AmeEur='e')
print(f"Θ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

Jarrow-Rudd:
Market Price Jarrow-Rudd: 132.2022
Delta: 0.4290
Gamma: -0.0000
Rho: -285.3329
Vega: 62.4383
Theta: 7.1489
Δ: Taylor Approximation: 132.2043524976; Exact price: 132.4167172434;
𝜌: Taylor Approximation: 132.1736741050; Exact price: 132.2002100988;
𝜗: Taylor Approximation: 132.8265907187; Exact price: 132.8249580816;
Γ: Taylor Approximation: 132.2043524976; Exact price: 132.4167172434;
Θ: Taylor Approximation: 132.2220653195; Exact price: 132.2220653195;


In [ ]:
# Market Price
MKTPrice = LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
print(f'Haahtela:')
print(f"Market Price: {MKTPrice:.4f}")

# Calculate Greeks
delta = calculate_delta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
gamma = calculate_gamma(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
rho = calculate_rho(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
vega = calculate_vega(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
theta = calculate_theta(S0, F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Rho: {rho:.4f}")
print(f"Vega: {vega:.4f}")  
print(f"Theta: {theta:.4f}")

bump=0.5/100
Taylor_approx=MKTPrice + delta * bump
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
print(f"Δ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/10000
Taylor_approx = MKTPrice + rho * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r*(1+bump), k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
print(f"𝜌:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/100
Taylor_approx = MKTPrice + vega * bump
Exact_price=LatticeConvertibleBond(S0, F, T, r, k, coupon, conversion_ratio, N, sigma+bump, model_type='Haahtela', AmeEur='e')
print(f"𝜗:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=0.5/100
Taylor_approx = MKTPrice + delta * bump + 0.5 * gamma * (bump) ** 2
Exact_price=LatticeConvertibleBond(S0*(1+bump), F, T, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
print(f"Γ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

bump=1/360
Taylor_approx=MKTPrice + theta * bump
Exact_price=LatticeConvertibleBond(S0, F, T-bump, r, k, coupon, conversion_ratio, N, sigma, model_type='Haahtela', AmeEur='e')
print(f"Θ:",f"Taylor Approximation: {Taylor_approx:.10f};",f"Exact price: {Exact_price:.10f};")

Haahtela:
Market Price: 132.2086
Delta: 0.4291
Gamma: 0.0000
Rho: -285.3329
Vega: 62.5352
Theta: 7.1470
Δ: Taylor Approximation: 132.2107577757; Exact price: 132.4231542261;
𝜌: Taylor Approximation: 132.1800790629; Exact price: 132.2066150567;
𝜗: Taylor Approximation: 132.8339641035; Exact price: 132.8323795977;
Γ: Taylor Approximation: 132.2107577757; Exact price: 132.4231542261;
Θ: Taylor Approximation: 132.2284650351; Exact price: 132.2284650351;
